# Precomputing values for use in fits of Stan models

Because of the way Stan works, it is necessary to compute some values in advance which can then be passed into the fit an interpolated over. The precomputed values will be different for different sets of source distances, and therefore different catalogues. 

Here we show how to compute the values for the SBG catalogue, but it is exactly the same for all cases, just changing the input label.

For ease, all the precomputed table files used are provided for use in this repository.

In [1]:
import os
import h5py

from fancy import Data, Model, Analysis
import numpy as np

# detector_type = "PAO"
detector_type = "TA"

In [2]:
'''set detector and detector properties'''
if detector_type == "TA":
    from fancy.detector.TA2015 import detector_properties, Eth
elif detector_type == "PAO":
    from fancy.detector.auger2014 import detector_properties, Eth
else:
    raise Exception("Undefined detector type!")


In [3]:
# Define file containing catalogue information
source_file = '../../data/sourcedata.h5'

# Path to Stan files
stan_path = '../../stan/'

# make output directory if it doesnt exist
if not os.path.isdir("output"):
    os.mkdir("output")

# source_type = "2FHL_250Mpc"
# source_type = "swift_BAT_213"
source_types = ["SBG_23", "2FHL_250Mpc"]

# exposure factor
# exp_factors = [1., 5., 10., 100.]
exp_factors = [1., 5.]
# exp_factors = [1.]
# exp_factor = 5.

# store original total exposure and area to restore per iteration
alpha_T_0 = detector_properties["alpha_T"]
A_0 = detector_properties["A"]

for source_type in source_types:
    for exp_factor in exp_factors:
        print("Current Source: {0}".format(source_type))
        print("Current Exposure Factor: {0}".format(exp_factor))

        # File in which to store precomputation
        # create new files with TA label
        # table_file = 'output/precomputation_storage.h5'
        # table_file = 'output/precomputation_storage_TA.h5'
        table_file = "output/tables_{0}_{2}_epsx{1:.0f}.h5".format(source_type, exp_factor, detector_type)

        # modify exposure factor
        detector_properties["alpha_T"] *= exp_factor
        detector_properties["A"] *= exp_factor

        # check if we have the correct values for detector_properties
        # if detector_properties["alpha_T"] / exp_factor, alpha_T_0):
        #     raise Exception("Fix exposure factor for detector property!")
        # if detector_properties["A"] / exp_factor, A_0):
        #     raise Exception("Fix exposure factor for detector property!")

        print(detector_properties["alpha_T"] / exp_factor == alpha_T_0)
        print(detector_properties["A"] / exp_factor == A_0)

        data = Data()
        data.add_source(source_file, source_type) 
        # data.source.select_sources([0, 1]) # just select 2 sources to speed up demo
        data.add_detector(detector_properties)  # KW: add detector information

        model_name = 'joint_model.stan'
        model = Model(model_filename = model_name, include_paths = stan_path)
        model.input(Eth = Eth) # EeV
        # model.input(Eth = 57) # EeV

        # precomp_output = 'output/testing_precomputation.h5'
        # precomp_output = 'output/testing_precomputation_TA.h5'
        # summary = b'Demonstration of precomputation.' 
        summary = b'Precomputation for SBG catalogue with TA observatory'
        # summary = b'Precomputation for 2FHL catalogue with TA observatory'
        # summary = b'Precomputation for swift_bat catalogue with TA observatory'
        analysis = Analysis(data, model, analysis_type = 'joint', 
                            filename = table_file, summary = summary)

        analysis.build_tables(fit_only = True)
        analysis.tables.save(table_file)

        analysis.build_energy_table(table_file = table_file)

        # reset exposure factor to original
        detector_properties["alpha_T"] = alpha_T_0
        detector_properties["A"] = A_0

Current Source: SBG_23
Current Exposure Factor: 1.0


Precomputing exposure integral: 100%|██████████| 23/23 [01:45<00:00,  4.59s/it]


Precomputing exposure integral: 100%|██████████| 23/23 [01:01<00:00,  2.66s/it]


Current Source: SBG_23
Current Exposure Factor: 5.0


Precomputing exposure integral:   0%|          | 0/23 [00:05<?, ?it/s]Process ForkPoolWorker-56:
Process ForkPoolWorker-66:
Process ForkPoolWorker-67:
Process ForkPoolWorker-69:
Process ForkPoolWorker-60:
Process ForkPoolWorker-54:
Process ForkPoolWorker-57:
Process ForkPoolWorker-65:
Process ForkPoolWorker-51:
Process ForkPoolWorker-62:
Process ForkPoolWorker-59:
Process ForkPoolWorker-55:
Process ForkPoolWorker-53:
Process ForkPoolWorker-61:
Process ForkPoolWorker-52:
Traceback (most recent call last):
Process ForkPoolWorker-70:
Process ForkPoolWorker-71:
Process ForkPoolWorker-58:
Process ForkPoolWorker-68:
Traceback (most recent call last):
Process ForkPoolWorker-63:
Process ForkPoolWorker-64:
Process ForkPoolWorker-72:
Traceback (most recent call last):



KeyboardInterrupt: 